In [1]:

import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train = pd.read_csv('/content/drive/My Drive/Data/KDDtrain_updated.csv')
train.head()

,Unnamed: 0,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack
0,0,0,1,20,9,491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,150,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,0
1,1,0,2,44,9,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,0
2,2,0,1,49,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,1
3,3,0,1,24,9,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,0
4,4,0,1,24,9,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0


In [4]:
train.shape

(125973, 42)

In [7]:
test = pd.read_csv('/content/drive/My Drive/Data/KDDtest_updated.csv')
test.head()

,Unnamed: 0,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack
0,0,0,1,45,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,229,10,0.0,0.00,1.0,1.0,0.04,0.06,0.00,255,10,0.04,0.06,0.00,0.00,0.0,0.0,1.00,1.00,1
1,1,0,1,45,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,136,1,0.0,0.00,1.0,1.0,0.01,0.06,0.00,255,1,0.00,0.06,0.00,0.00,0.0,0.0,1.00,1.00,1
2,2,2,1,19,9,12983,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.00,0.0,0.0,1.00,0.00,0.00,134,86,0.61,0.04,0.61,0.02,0.0,0.0,0.00,0.00,0
3,3,0,0,13,9,20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,65,0.0,0.00,0.0,0.0,1.00,0.00,1.00,3,57,1.00,0.00,1.00,0.28,0.0,0.0,0.00,0.00,2
4,4,1,1,55,2,0,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,8,0.0,0.12,1.0,0.5,1.00,0.00,0.75,29,86,0.31,0.17,0.03,0.02,0.0,0.0,0.83,0.71,2


In [6]:
test.shape

(22544, 42)

In [8]:
train = train.drop(columns="Unnamed: 0")
test = test.drop(columns='Unnamed: 0')

In [9]:
train.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack
0,0,1,20,9,491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,150,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,0
1,0,2,44,9,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,0
2,0,1,49,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,1
3,0,1,24,9,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,0
4,0,1,24,9,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0


In [10]:
test.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack
0,0,1,45,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,229,10,0.0,0.00,1.0,1.0,0.04,0.06,0.00,255,10,0.04,0.06,0.00,0.00,0.0,0.0,1.00,1.00,1
1,0,1,45,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,136,1,0.0,0.00,1.0,1.0,0.01,0.06,0.00,255,1,0.00,0.06,0.00,0.00,0.0,0.0,1.00,1.00,1
2,2,1,19,9,12983,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.00,0.0,0.0,1.00,0.00,0.00,134,86,0.61,0.04,0.61,0.02,0.0,0.0,0.00,0.00,0
3,0,0,13,9,20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,65,0.0,0.00,0.0,0.0,1.00,0.00,1.00,3,57,1.00,0.00,1.00,0.28,0.0,0.0,0.00,0.00,2
4,1,1,55,2,0,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,8,0.0,0.12,1.0,0.5,1.00,0.00,0.75,29,86,0.31,0.17,0.03,0.02,0.0,0.0,0.83,0.71,2


In [49]:
X_train=train[train.columns.difference(['attack'])]
y_train=train['attack']

In [50]:
X_train.head()

,count,diff_srv_rate,dst_bytes,dst_host_count,dst_host_diff_srv_rate,dst_host_rerror_rate,dst_host_same_src_port_rate,dst_host_same_srv_rate,dst_host_serror_rate,dst_host_srv_count,dst_host_srv_diff_host_rate,dst_host_srv_rerror_rate,dst_host_srv_serror_rate,duration,flag,hot,is_guest_login,is_host_login,land,logged_in,num_access_files,num_compromised,num_failed_logins,num_file_creations,num_root,num_shells,protocol_type,rerror_rate,root_shell,same_srv_rate,serror_rate,service,src_bytes,srv_count,srv_diff_host_rate,srv_rerror_rate,srv_serror_rate,su_attempted,urgent,wrong_fragment
0,2,0.00,0,150,0.03,0.05,0.17,0.17,0.00,25,0.00,0.00,0.00,0,9,0,0,0,0,0,0,0,0,0,0,0,1,0.0,0,1.00,0.0,20,491,2,0.00,0.0,0.0,0,0,0
1,13,0.15,0,255,0.60,0.00,0.88,0.00,0.00,1,0.00,0.00,0.00,0,9,0,0,0,0,0,0,0,0,0,0,0,2,0.0,0,0.08,0.0,44,146,1,0.00,0.0,0.0,0,0,0
2,123,0.07,0,255,0.05,0.00,0.00,0.10,1.00,26,0.00,0.00,1.00,0,5,0,0,0,0,0,0,0,0,0,0,0,1,0.0,0,0.05,1.0,49,0,6,0.00,0.0,1.0,0,0,0
3,5,0.00,8153,30,0.00,0.00,0.03,1.00,0.03,255,0.04,0.01,0.01,0,9,0,0,0,0,1,0,0,0,0,0,0,1,0.0,0,1.00,0.2,24,232,5,0.00,0.0,0.2,0,0,0
4,30,0.00,420,255,0.00,0.00,0.00,1.00,0.00,255,0.00,0.00,0.00,0,9,0,0,0,0,1,0,0,0,0,0,0,1,0.0,0,1.00,0.0,24,199,32,0.09,0.0,0.0,0,0,0


In [51]:
y_train.head()

0    0
1    0
2    1
3    0
4    0
Name: attack, dtype: int64

In [52]:
X_test=test[test.columns.difference(['attack'])]
y_test=test['attack']

# 5-fold Cross validation 

In [21]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
import numpy as np

### Logistic Regression

In [25]:
pipe_lr = Pipeline([('scl', StandardScaler()),('clf', LogisticRegression(penalty='l2', random_state=0))])

In [53]:
train_sizes, train_scores, test_scores = learning_curve(estimator=pipe_lr,X=X_train, y=y_train, train_sizes=np.linspace(0.2,1.0,5), cv=5, n_jobs=-1)

In [55]:
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

In [56]:
for i in train_sizes:
    print(i)

20155
40311
60466
80622
100778


In [57]:
for i in train_mean:
    print(i)

0.9660729347556437
0.9659050879412566
0.9659411900902987
0.966098583513185
0.9660005159856319


In [58]:

for i in test_mean:
    print(i)

0.9651592097791865
0.9652862257230783
0.965413228118603
0.9657386907799879
0.9655481783645081


## SVM

In [59]:
from sklearn.svm import LinearSVC

In [60]:
pipe_svc = Pipeline([('scl', StandardScaler()),('clf', LinearSVC())])

In [61]:
train_sizes_svc, train_scores_svc, test_scores_svc = learning_curve(estimator=pipe_svc,X=X_train, y=y_train, train_sizes=np.linspace(0.2,1.0,5), cv=5, n_jobs=-1)
train_mean_svc = np.mean(train_scores_svc, axis=1)
train_std_svc = np.std(train_scores_svc, axis=1)
test_mean_svc = np.mean(test_scores_svc, axis=1)
test_std_svc = np.std(test_scores_svc, axis=1)

In [62]:
for i in train_mean_svc:
    print(i)

0.9606747705284049
0.9601746421572275
0.9607316508451029
0.9606038054129146
0.9608724126297407


In [63]:
for i in test_mean_svc:
    print(i)

0.9600152349815442
0.9596500954537618
0.9604836007804678
0.9600231714891055
0.9605867901874469


## Decision Tree - 5 fold KFCV

In [64]:
from sklearn import tree

In [65]:
pipe_tree = Pipeline([('scl', StandardScaler()),('clf', tree.DecisionTreeClassifier())])

In [66]:
train_sizes_tree, train_scores_tree, test_scores_tree = learning_curve(estimator=pipe_tree,X=X_train, y=y_train, train_sizes=np.linspace(0.2,1.0,5), cv=5, n_jobs=-1)
train_mean_tree = np.mean(train_scores_tree, axis=1)
train_std_tree = np.std(train_scores_tree, axis=1)
test_mean_tree = np.mean(test_scores_tree, axis=1)
test_std_tree = np.std(test_scores_tree, axis=1)

In [67]:
for i in train_mean_tree:
    print(i)

0.9999603076159762
0.9999652700255515
0.9999702312043132
0.9999553471757089
0.9999543551171882


In [68]:
for i in test_mean_tree:
    print(i)

0.994562331256857
0.9964039970645413
0.996959666967267
0.997229568089234
0.9975788453145563


## Random Forest 5 fold CV

In [69]:
from sklearn.ensemble import RandomForestClassifier

In [70]:
pipe_rnd = Pipeline([('scl', StandardScaler()),('clf', RandomForestClassifier(n_estimators=10))])

In [71]:
train_sizes_rnd, train_scores_rnd, test_scores_rnd = learning_curve(estimator=pipe_rnd,X=X_train, y=y_train, train_sizes=np.linspace(0.2,1.0,5), cv=5, n_jobs=-1)
train_mean_rnd = np.mean(train_scores_rnd, axis=1)
train_std_rnd = np.std(train_scores_rnd, axis=1)
test_mean_rnd = np.mean(test_scores_rnd, axis=1)
test_std_rnd = np.std(test_scores_rnd, axis=1)

In [72]:
for i in train_mean_rnd:
    print(i)

0.999781691887869
0.9997420059040957
0.9998313101577747
0.999851157252363
0.9998868800730317


In [73]:
for i in test_mean_rnd:
    print(i)

0.9967850344986331
0.9977058647243096
0.9981186475584904
0.9983567928827842
0.998539371311329


## ANN 5 Fold CV

In [74]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(50,), max_iter=10, alpha=1e-4, solver='sgd', verbose=10, tol=1e-4, random_state=1, learning_rate_init=.1)

In [75]:
pipe_mlp = Pipeline([('scl', StandardScaler()),('clf', mlp)])

In [76]:
train_sizes_mlp, train_scores_mlp, test_scores_mlp = learning_curve(estimator=pipe_mlp,X=X_train, y=y_train, train_sizes=np.linspace(0.2,1.0,5), cv=5, n_jobs=-1)
train_mean_mlp = np.mean(train_scores_mlp, axis=1)
train_std_mlp = np.std(train_scores_mlp, axis=1)
test_mean_mlp = np.mean(test_scores_mlp, axis=1)
test_std_mlp = np.std(test_scores_mlp, axis=1)

In [77]:
for i in train_mean_mlp:
    print(i)

0.9929942942197967
0.9930936965096375
0.993679092382496
0.9941604028677036
0.9940026593105639


In [78]:
for i in test_mean_mlp:
    print(i)

0.9912917872018436
0.9922284809902937
0.9928317979803575
0.9936256223446479
0.99314138195521


# Evaluation

## Logistic Regression

In [ ]:
pipe_lr = pipe_lr.fit(X_train, y_train)

In [80]:
y_pred_train = pipe_lr.predict(X_train)

In [117]:
y_pred_test = pipe_lr.predict(X_test)

In [82]:
from sklearn.metrics import accuracy_score

In [118]:
accuracy_score(y_train, y_pred_train), accuracy_score(y_test, y_pred_test)

(0.9953243949100204, 0.7598917672107878)

In [84]:
accuracy_score(y_test, y_pred_test)

0.7598917672107878

In [85]:
from sklearn.metrics import classification_report

In [86]:
target_names = ['Normal', 'DoS','Probe','R2L', 'U2R']

In [87]:
print(classification_report(y_train, y_pred_train, target_names=target_names))

              precision    recall  f1-score   support

      Normal       0.97      0.97      0.97     67343
         DoS       0.99      0.99      0.99     45927
       Probe       0.89      0.85      0.87     11656
         R2L       0.73      0.67      0.70       995
         U2R       0.79      0.37      0.50        52

    accuracy                           0.97    125973
   macro avg       0.87      0.77      0.81    125973
weighted avg       0.97      0.97      0.97    125973



In [88]:
print(classification_report(y_test, y_pred_test, target_names=target_names))

              precision    recall  f1-score   support

      Normal       0.67      0.95      0.79      9900
         DoS       0.94      0.82      0.88      7165
       Probe       0.84      0.74      0.79      2421
         R2L       0.45      0.00      0.01      2991
         U2R       0.85      0.16      0.28        67

    accuracy                           0.76     22544
   macro avg       0.75      0.54      0.55     22544
weighted avg       0.74      0.76      0.71     22544



In [89]:
from sklearn.metrics import confusion_matrix
import itertools

In [90]:
cnf_matrix = confusion_matrix(y_test, y_pred_test)

In [91]:
cnf_matrix

array([[9439,  283,  166,   10,    2],
       [1155, 5877,  133,    0,    0],
       [ 532,   94, 1795,    0,    0],
       [2940,    0,   42,    9,    0],
       [  54,    0,    1,    1,   11]])

## SVM

In [92]:
pipe_svc = pipe_svc.fit(X_train, y_train)
y_pred_train = pipe_svc.predict(X_train)
y_pred_test = pipe_svc.predict(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [93]:
accuracy_score(y_train, y_pred_train), accuracy_score(y_test, y_pred_test)

(0.960039055988188, 0.755722143364088)

In [94]:
print(classification_report(y_train, y_pred_train, target_names=target_names))

              precision    recall  f1-score   support

      Normal       0.95      0.97      0.96     67343
         DoS       0.99      0.99      0.99     45927
       Probe       0.88      0.83      0.86     11656
         R2L       0.70      0.33      0.45       995
         U2R       0.51      0.52      0.51        52

    accuracy                           0.96    125973
   macro avg       0.81      0.73      0.75    125973
weighted avg       0.96      0.96      0.96    125973



In [95]:
print(classification_report(y_test, y_pred_test, target_names=target_names))

              precision    recall  f1-score   support

      Normal       0.67      0.95      0.78      9900
         DoS       0.94      0.82      0.88      7165
       Probe       0.81      0.70      0.75      2421
         R2L       0.62      0.01      0.01      2991
         U2R       0.75      0.36      0.48        67

    accuracy                           0.76     22544
   macro avg       0.76      0.57      0.58     22544
weighted avg       0.76      0.76      0.71     22544



In [98]:
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.67      0.95      0.78      9900
           1       0.94      0.82      0.88      7165
           2       0.81      0.70      0.75      2421
           3       0.62      0.01      0.01      2991
           4       0.75      0.36      0.48        67

    accuracy                           0.76     22544
   macro avg       0.76      0.57      0.58     22544
weighted avg       0.76      0.76      0.71     22544



In [96]:
cnf_matrix = confusion_matrix(y_test, y_pred_test)

In [97]:
cnf_matrix

array([[9424,  275,  187,    8,    6],
       [1157, 5885,  123,    0,    0],
       [ 627,  108, 1686,    0,    0],
       [2881,    3,   87,   18,    2],
       [  35,    1,    4,    3,   24]])

## Decision tree

In [99]:
pipe_tree = pipe_tree.fit(X_train, y_train)
y_pred_train = pipe_tree.predict(X_train)
y_pred_test = pipe_tree.predict(X_test)

In [100]:
accuracy_score(y_train, y_pred_train), accuracy_score(y_test, y_pred_test)

(0.999944432537131, 0.7676987224982257)

In [101]:
print(classification_report(y_train, y_pred_train, target_names=target_names))
#print(classification_report(y_train, y_pred_train))

              precision    recall  f1-score   support

      Normal       1.00      1.00      1.00     67343
         DoS       1.00      1.00      1.00     45927
       Probe       1.00      1.00      1.00     11656
         R2L       1.00      1.00      1.00       995
         U2R       1.00      0.98      0.99        52

    accuracy                           1.00    125973
   macro avg       1.00      1.00      1.00    125973
weighted avg       1.00      1.00      1.00    125973



In [102]:
print(classification_report(y_test, y_pred_test, target_names=target_names))

              precision    recall  f1-score   support

      Normal       0.72      0.97      0.83      9900
         DoS       0.96      0.86      0.90      7165
       Probe       0.55      0.61      0.58      2421
         R2L       0.81      0.04      0.07      2991
         U2R       0.67      0.18      0.28        67

    accuracy                           0.77     22544
   macro avg       0.74      0.53      0.53     22544
weighted avg       0.79      0.77      0.72     22544



In [103]:
cnf_matrix = confusion_matrix(y_test, y_pred_test)
cnf_matrix

array([[9561,   70,  244,   22,    3],
       [ 355, 6144,  666,    0,    0],
       [ 735,  211, 1475,    0,    0],
       [2570,    0,  303,  115,    3],
       [  50,    0,    0,    5,   12]])

## Random Forest

In [104]:
pipe_rnd = pipe_rnd.fit(X_train, y_train)
y_pred_train = pipe_rnd.predict(X_train)
y_pred_test = pipe_rnd.predict(X_test)

In [105]:
y_pred_train = pipe_rnd.predict(X_train)
y_pred_test = pipe_rnd.predict(X_test)

In [106]:
accuracy_score(y_train, y_pred_train), accuracy_score(y_test, y_pred_test)

(0.9998571122383367, 0.7544801277501775)

In [107]:
print(classification_report(y_train, y_pred_train, target_names=target_names))

              precision    recall  f1-score   support

      Normal       1.00      1.00      1.00     67343
         DoS       1.00      1.00      1.00     45927
       Probe       1.00      1.00      1.00     11656
         R2L       1.00      0.99      1.00       995
         U2R       1.00      0.94      0.97        52

    accuracy                           1.00    125973
   macro avg       1.00      0.99      0.99    125973
weighted avg       1.00      1.00      1.00    125973



In [108]:
print(classification_report(y_test, y_pred_test, target_names=target_names))

              precision    recall  f1-score   support

      Normal       0.66      0.97      0.79      9900
         DoS       0.96      0.79      0.87      7165
       Probe       0.85      0.64      0.73      2421
         R2L       0.99      0.05      0.09      2991
         U2R       1.00      0.03      0.06        67

    accuracy                           0.75     22544
   macro avg       0.89      0.50      0.51     22544
weighted avg       0.82      0.75      0.71     22544



In [109]:
cnf_matrix = confusion_matrix(y_test, y_pred_test)
cnf_matrix

array([[9649,   58,  192,    1,    0],
       [1418, 5676,   71,    0,    0],
       [ 713,  170, 1538,    0,    0],
       [2830,    0,   17,  144,    0],
       [  65,    0,    0,    0,    2]])

## ANN

In [110]:
pipe_mlp = pipe_mlp.fit(X_train, y_train)
y_pred_train = pipe_mlp.predict(X_train)
y_pred_test = pipe_mlp.predict(X_test)

Iteration 1, loss = 0.07196939
Iteration 2, loss = 0.03071902
Iteration 3, loss = 0.02706321
Iteration 4, loss = 0.02479211
Iteration 5, loss = 0.02301724
Iteration 6, loss = 0.02229233
Iteration 7, loss = 0.02066422
Iteration 8, loss = 0.02217405
Iteration 9, loss = 0.02035659
Iteration 10, loss = 0.01923917


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [111]:
accuracy_score(y_train, y_pred_train), accuracy_score(y_test, y_pred_test)

(0.9953243949100204, 0.7755943931866572)

In [112]:
print(classification_report(y_train, y_pred_train, target_names=target_names))

              precision    recall  f1-score   support

      Normal       0.99      1.00      1.00     67343
         DoS       1.00      1.00      1.00     45927
       Probe       0.99      0.99      0.99     11656
         R2L       0.87      0.86      0.86       995
         U2R       0.78      0.35      0.48        52

    accuracy                           1.00    125973
   macro avg       0.93      0.84      0.87    125973
weighted avg       1.00      1.00      1.00    125973



In [113]:
print(classification_report(y_test, y_pred_test, target_names=target_names))

              precision    recall  f1-score   support

      Normal       0.69      0.98      0.81      9900
         DoS       0.95      0.85      0.90      7165
       Probe       0.81      0.70      0.75      2421
         R2L       0.85      0.02      0.03      2991
         U2R       0.71      0.15      0.25        67

    accuracy                           0.78     22544
   macro avg       0.80      0.54      0.55     22544
weighted avg       0.81      0.78      0.73     22544



In [114]:
cnf_matrix = confusion_matrix(y_test, y_pred_test)

In [115]:
cnf_matrix

array([[9674,   55,  163,    5,    3],
       [ 962, 6069,  134,    0,    0],
       [ 485,  251, 1685,    0,    0],
       [2849,    2,   92,   47,    1],
       [  50,    4,    0,    3,   10]])